# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122681e+02     1.492872e+00
 * time: 0.5708000659942627
     1     1.148303e+01     9.758391e-01
 * time: 1.9773430824279785
     2    -1.184628e+01     1.035129e+00
 * time: 2.5637030601501465
     3    -3.401799e+01     7.782094e-01
 * time: 3.4736480712890625
     4    -4.774517e+01     5.633767e-01
 * time: 4.315453052520752
     5    -5.689684e+01     1.888836e-01
 * time: 5.156620979309082
     6    -5.977584e+01     1.038155e-01
 * time: 5.722622871398926
     7    -6.084505e+01     5.311621e-02
 * time: 6.274955987930298
     8    -6.122395e+01     7.311219e-02
 * time: 6.838320970535278
     9    -6.152661e+01     3.353111e-02
 * time: 7.413519859313965
    10    -6.172266e+01     2.818243e-02
 * time: 7.965532064437866
    11    -6.189835e+01     2.387256e-02
 * time: 8.51366901397705
    12    -6.197552e+01     1.884651e-02
 * time: 9.072512865066528
    13    -6.206069e+01     1.733363e-02
 * time: 9.623475074768066
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557701
    AtomicNonlocal      14.8522659
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336825

    total               -62.261666461227
